In [1]:
from gensim.models import Word2Vec
import pandas as pd
import pickle
import time
import logging
import multiprocessing as mp
logging.basicConfig(
	format='%(asctime)s : %(levelname)s : %(message)s',
	level=logging.INFO)

cores = mp.cpu_count
list_corpus = ['corpus/attraction_tag.list',
               'corpus/hotel_tag.list',
               'corpus/restaurant_tag.list']
list_tag_model = ['model/attraction_tag.model',
                   'model/hotel_tag.model',
                   'model/restaurant_tag.model']
params_user = [{'size':300, 'window':99999, 'min_count':0,        # Attraction
               'workers':cores, 'iter':100, 'sg':1, 'sample':1},
               {'size':300, 'window':99999, 'min_count':0,        # Hotel
               'workers':cores, 'iter':100, 'sg':1, 'sample':1},
               {'size':300, 'window':99999, 'min_count':0,        # Restaurant
               'workers':cores, 'iter':100, 'sg':1, 'sample':1}]

C:\Users\CypressRH\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [19]:
with open(list_corpus[2], 'rb') as f:
    corpus = pickle.load(f)

In [20]:
len(corpus)

221

In [ ]:
spent = []
for i in range(3):
    with open(list_corpus[i], 'rb') as f:
        corpus = pickle.load(f)
    start = time.time()
    model = Word2Vec(corpus, **params_user[i])
    spent.append('Elapsed time: '+str(time.time() - start)+' sec'+' ['+list_user_model[i]+']')
    model.wv.save(list_user_model[i])